In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [55]:
arm_mass = 0.2 #kg
arm_length = 0.6 #meter
motor_mass = 0.065

arm_lengths = [arm_length,]*4

total_inertia_tensor = np.zeros((3, 3))

arms_inertia_tensor = np.zeros((3, 3))
arms_inertia_tensor[0, 0] = 2*(1/12)*arm_mass*arm_length**2
arms_inertia_tensor[1, 1] = (1/12)*arm_mass*arm_length**2
arms_inertia_tensor[2, 2] = (1/12)*arm_mass*arm_length**2

motors_inertia_tensor = np.zeros((3, 3))
motors_inertia_tensor[0, 0] = 2*motor_mass*arm_length**2
motors_inertia_tensor[1, 1] = 2*motor_mass*arm_length**2
motors_inertia_tensor[2, 2] = 4*motor_mass*arm_length**2

In [56]:
#rot_z_45 = np.zeros((3, 3))
#rot_z_45[0, 0] = np.cos(np.pi/4)
#rot_z_45[0, 1] = -np.sin(np.pi/4)
#rot_z_45[1, 0] = np.sin(np.pi/4)
#rot_z_45[1, 1] = np.cos(np.pi/4)
#rot_z_45[2, 2] = 1

total_inertia_tensor = arms_inertia_tensor + motors_inertia_tensor
#total_inertia_tensor = rot_z_45.dot(total_inertia_tensor)

In [62]:
def quat_mult(q1, q2):
    q3 = np.zeros((4, 1))
    q3[0] = q1[0]*q2[0] - q1[1:].dot(q2[1:])
    q3[1:] = q1[0]*q2[1:] + q2[0]*q1[1:] + np.cross(q1[1:], q2[1:])
    return q3

In [66]:
def quat_inv(q1):
    q2 = np.zeros((4, 1))
    mag = np.linalg.norm(q1)
    q2[0] = q1[0]
    q2[1:] = -1.*q1[1:]
    q2 /= (mag*mag)
    return q2

In [57]:
def calc_omega_dot_body(omega_body, inertia_body, torques_body):
    omega_dot_body = np.linalg.inv(inertia_body).dot(torques_body - np.cross(omega_body, (inertia_body.dot(omega_body))))
    return omega_dot_body

In [59]:
def get_torques_body(motor_speeds, arm_lengths, k_force, k_torque):
    torques_body = np.zeros((3, 1))
    torques_body[0] = arm_lengths[1]*k_force*(motor_speeds[1]**2) - arm_lengths[2]*k_force*(motor_speeds[2]**2)
    torques_body[1] = arm_lengths[3]*k_force*(motor_speeds[3]**2) - arm_lengths[0]*k_force*(motor_speeds[0]**2)
    torques_body[2] = k_torque*(-1*motor_speeds[0]**2 + motor_speeds[1]**2 - motor_speeds[2]**2 + motor_speeds[3]**2)
    return torques_body

In [60]:
def get_forces_body(motor_speeds, k_force):
    forces_body = np.zeros((3, 1))
    forces_body[2] = k_force*sum([s**2 for s in motor_speeds])
    return forces_body

In [64]:
def update_orientation_body(omega_body_prev, q_body_prev, dt, arm_lengths, k_force, k_torque, inertia_body, motor_speeds):
    torques_body = get_torques_body(motor_speeds, arm_lengths, k_force, k_torque)
    omega_dot_body = calc_omega_dot_body(omega_body_prev, inertia, torques_body)
    omega_body = omega_body_prev + omega_dot_body*dt
    delta_theta = np.linalg.norm(omega_body)
    quat_omega_body = np.array([np.cos(delta_theta*dt/2.)] + list(np.sin(delta_theta*dt/2.)*omega_body/delta_theta))
    q_body = quat_mult(q_body_prev, quat_omega_body)
    if np.linalg.norm(q_body) < 0.999:
        q_body += 0.1*(1. - np.linalg.norm(q_body)**2)*q_body
        
    return q_body

In [67]:
def get_accel_inertial(vel_inert_prev, q_body, motor_speeds, k_force, k_damp):
    accel_inert = np.zeros((3, 1))
    forces_body = get_forces_body(motor_speeds, k_force)
    forces_body_quat = np.zeros((4, 1))
    forces_body_quat[1:] = forces_body
    accel_inert = np.array([0, 0, -9.8]) + \
        quat_mult(quat_mult(quat_inv(q_body), forces_body_quat), q_body)[1:] - \
        k_damp*vel_inert_prev
    return accel_inert

In [68]:
def update_pos_vel_inertial(pos_inert_prev, vel_inert_prev, accel_inert_prev, q_body, motor_speeds, k_force, k_damp, dt):
    
    # Velocity verlet
    pos_inert = pos_inert_prev + vel_inert_prev*dt + 0.5*accel_inert_prev*dt*dt
    accel_inert = get_accel_inertial(vel_inert_prev, q_body, motor_speeds, k_force, k_damp)
    vel_inert = vel_inert_prev + 0.5(accel_inert + accel_inert_prev)*dt
    
    return (vel_inert, pos_inert, accel_inert)